<a href="https://colab.research.google.com/github/tassiogomes/Phyton-maps-colaboratory/blob/master/Tassio_Projeto_Python_Maps_com_biblioteca_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tassio - Projeto Python Maps com biblioteca Folium

In [0]:
import pandas as pd
import folium
import numpy as np
import matplotlib.pyplot as plt

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

wb = gc.open_by_url ('https://docs.google.com/spreadsheets/d/1KDODUlsvnNlxzeJPXF5D5I4ed7YtQz9a7gKGjzGCRsE/edit?ts=5e700389#gid=0')
sheet = wb.worksheet('Dados')
data = sheet.get_all_values()

df_data = pd.DataFrame(data)
df_data.columns = df_data.iloc[0]
df_data = df_data.iloc[1:]
df_data

new_df2 = df_data.groupby(['Distrito'])["Distrito"].count().reset_index(name="Count")
new_df2

!wget --quiet https://raw.githubusercontent.com/ufoe/d3js-geojson/master/Portugal.json
world_geo = r'https://raw.githubusercontent.com/ufoe/d3js-geojson/master/Portugal.json' # geojson file
print('world_geo created!')

latitude = 41.14961 
longitude = -8.61099
Portugal = folium.Map(location=[latitude, longitude], zoom_start=8)
df_data = folium.map.FeatureGroup()
print('Data downloaded and read into a dataframe!')

threshold_scale = np.linspace(new_df2['Count'].min(),
                              new_df2['Count'].max(),
                              10, dtype=int)
threshold_scale = threshold_scale.tolist() 
threshold_scale[-1] = threshold_scale[-1] + 1 

Portugal.choropleth(
    geo_data=world_geo,
    data= new_df2,
    columns=['Distrito','Count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    threshold_scale=threshold_scale,
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name= 'Pessoas com possíveis sintomas',
    reset=True
)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    world_geo,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'region'],
        aliases=['District: ', 'Region'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
Portugal.add_child(NIL)
Portugal.keep_in_front(NIL)
folium.LayerControl().add_to(Portugal)

Portugal

world_geo created!
Data downloaded and read into a dataframe!


/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
